# Multifactor Scoring Model

The components of the S&P 500 Index are ranked by factors thought to be predictive of future returns. In this example, three factors are chosen: value, momentum, and profitability...represented by forward price/sales, one year total return, and return on invested capital. These factors are standardized and grouped by GICS sector classifications. The standardized statistic is winsorized to reduce outlier effects. Finally, the program generates a formatted Excel file of the results. This multifactor scoring model is for code demonstration purposes only and is not intended to be an investment recommendation or used to pick investments. 

In [1]:
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from pyfinlab import data_api as api

Enter your ticker universe in this Excel file (../data/multifactor_inputs.xlsx).

In [2]:
tickers = list(pd.read_excel('../data/multifactor_inputs.xlsx', engine='openpyxl', sheet_name='inputs').squeeze())

Download data. The data here requires a subscription but you can create your own dataset from whatever data source you have. 

In [3]:
data = api.current_equity_data(
    tickers, 
    info=['LONG_COMP_NAME', 'GICS_SECTOR_NAME','BEST_PX_SALES_RATIO', 'CURRENT_TRR_6MO', 'RETURN_ON_INV_CAPITAL'], 
    api_source='bloomberg'
).sort_index()
data = data.reset_index()
data = data.replace(' US Equity', '', regex=True).set_index('TICKER')
data

,LONG_COMP_NAME,GICS_SECTOR_NAME,BEST_PX_SALES_RATIO,CURRENT_TRR_6MO,RETURN_ON_INV_CAPITAL
TICKER,,,,,
A,Agilent Technologies Inc,Health Care,6.720270,8.567838,15.347929
AAL,American Airlines Group Inc,Industrials,0.256214,-31.604930,-6.471751
AAP,Advance Auto Parts Inc,Consumer Discretionary,1.283908,18.865410,9.502187
AAPL,Apple Inc,Information Technology,6.963313,28.937450,46.257607
ABBV,AbbVie Inc,Health Care,3.573838,8.224333,14.736852
...,...,...,...,...,...
YUM,Yum! Brands Inc,Consumer Discretionary,5.320417,5.917987,60.901504
ZBH,Zimmer Biomet Holdings Inc,Health Care,3.133220,-23.590470,4.819306
ZBRA,Zebra Technologies Corp,Information Technology,5.399640,13.883880,23.670112


Standardize the data grouped by sector, winsorize to reduce effects from outliers, and replace N/A values with the average sector value. 

In [4]:
zscore = lambda x: (x - x.mean()) / x.std(ddof=1)
zscores = data.copy().groupby(['GICS_SECTOR_NAME']).transform(zscore).clip(-3, 3).fillna(0).round(2)
zscores.columns = ['VALUE', 'MOMENTUM', 'PROFITABILITY']

Compute the multifactor scoring model.

In [5]:
zscores.insert(0, 'SCORE', ((
    - zscores['VALUE']
    + zscores['MOMENTUM']
    + zscores['PROFITABILITY']
) / len(zscores.columns)).round(2))
zscores = pd.concat([data.GICS_SECTOR_NAME, zscores], axis=1)
zscores.insert(1, 'RANK', zscores[['SCORE', 'GICS_SECTOR_NAME']].groupby(['GICS_SECTOR_NAME']).rank(pct=True).round(2))
zscores.insert(0, 'NAME', data[['LONG_COMP_NAME']])

Show the top and bottom stocks for each sector. 

In [6]:
print('Top Ranked Stocks')
zscores[zscores['RANK']==1].sort_values(by='SCORE', ascending=False).head(11)

Top Ranked Stocks


,NAME,GICS_SECTOR_NAME,RANK,SCORE,VALUE,MOMENTUM,PROFITABILITY
TICKER,,,,,,,
MRNA,Moderna Inc,Health Care,1.0,1.78,0.04,2.39,3.00
NRG,NRG Energy Inc,Utilities,1.0,1.78,-1.51,0.84,3.00
HPQ,HP Inc,Information Technology,1.0,1.51,-1.25,0.55,2.73
WY,Weyerhaeuser Co,Real Estate,1.0,1.48,-1.39,0.05,3.00
MAS,Masco Corp,Industrials,1.0,1.45,-0.54,0.97,2.83
NUE,Nucor Corp,Materials,1.0,1.40,-0.95,0.34,2.92
COST,Costco Wholesale Corp,Consumer Staples,1.0,1.39,-0.80,3.00,0.36
DPZ,Domino's Pizza Inc,Consumer Discretionary,1.0,1.05,0.60,0.81,2.95
AMP,Ameriprise Financial Inc,Financials,1.0,1.02,-0.47,0.84,1.76


In [7]:
print('Bottom Ranked Stocks')
worst_stocks = []
for sector in list(zscores.GICS_SECTOR_NAME.drop_duplicates().values):
    worst_stocks.append(zscores[zscores['GICS_SECTOR_NAME']==sector].sort_values(by='RANK').head(1))
pd.concat(worst_stocks).sort_values(by='SCORE')

Bottom Ranked Stocks


,NAME,GICS_SECTOR_NAME,RANK,SCORE,VALUE,MOMENTUM,PROFITABILITY
TICKER,,,,,,,
NCLH,Norwegian Cruise Line Holdings Ltd,Consumer Discretionary,0.02,-1.23,-0.49,-2.00,-2.17
ALL,Allstate Corp/The,Financials,0.02,-1.19,-0.85,-1.83,-2.60
ILMN,Illumina Inc,Health Care,0.02,-1.18,1.43,-1.23,-0.88
TWTR,Twitter Inc,Communication Services,0.04,-1.18,0.89,-1.22,-1.44
KSU,Kansas City Southern,Industrials,0.01,-1.06,1.97,-0.25,-0.96
BF/B,Brown-Forman Corp,Consumer Staples,0.03,-1.01,2.82,-0.43,0.21
NEM,Newmont Corp,Materials,0.04,-0.88,0.54,-1.27,-0.84
V,Visa Inc,Information Technology,0.01,-0.84,1.39,-0.99,-0.15
HES,Hess Corp,Energy,0.05,-0.78,1.11,-1.08,-0.16


Save multifactor scoring model results as an Excel file. 

In [8]:
report_description = 'multifactor_zscores'
today = datetime.today().strftime('%m-%d-%Y')
filename = '../excel/{}_{}.xlsx'.format(report_description, today)

# Create mew Excel file.
writer = pd.ExcelWriter(filename, engine='xlsxwriter')

# Add worksheets in the order you want them here.
zscores.sort_values(by='RANK', ascending=False).to_excel(writer, sheet_name='zscores')

# Create workbook objects
workbook = writer.book
worksheet1 = writer.sheets['zscores']

# Workbook Formats
format1 = workbook.add_format({'fg_color': '#F2F2F2'})  # Background Color Left Aligned
format1.set_align('left')
format2 = workbook.add_format({'num_format': '0.00%',  # Percentage Style
                               'fg_color': '#F2F2F2'})
format3 = workbook.add_format({'num_format': '0.0000',  # Decimal Style
                               'fg_color': '#F2F2F2'})
format4 = workbook.add_format({'fg_color': '#F2F2F2'})  # Background Color
format5 = workbook.add_format({'num_format': '0.0000',  # Decimal Style with Border
                               'fg_color': '#F2F2F2'})
format5.set_left(2)

# factor_scores
worksheet1.hide_gridlines()
worksheet1.freeze_panes(1, 1)
worksheet1.set_column('A:A', 15.86, format1)
worksheet1.set_column('B:B', 40, format1)
worksheet1.set_column('C:C', 32, format1)
worksheet1.set_column('D:D', 23, format1)
worksheet1.set_column('E:E', 23, format1)
worksheet1.set_column('F:F', 23, format1)
worksheet1.set_column('G:G', 23, format1)
worksheet1.set_column('H:H', 23, format1)
worksheet1.set_column('I:XFD', None, None, {'hidden': True})
worksheet1.autofilter('A1:I{}'.format(zscores.shape[1] + 1))
worksheet1.set_default_row(hide_unused_rows=True)
worksheet1.conditional_format('D2:D{}'.format(zscores.shape[0] + 1), {'type': '3_color_scale'})
worksheet1.conditional_format('E2:E{}'.format(zscores.shape[0] + 1), {'type': '3_color_scale'})
worksheet1.conditional_format('F2:F{}'.format(zscores.shape[0] + 1), {'type': '3_color_scale', 
                                                                      'min_color': '#63BE7B',
                                                                      'mid_color': '#FFEB84',
                                                                      'max_color': '#F8696B'})
worksheet1.conditional_format('G2:G{}'.format(zscores.shape[0] + 1), {'type': '3_color_scale'})
worksheet1.conditional_format('H2:H{}'.format(zscores.shape[0] + 1), {'type': '3_color_scale'})

writer.save()